In [1]:
import rasterio
from pathlib import Path
import geopandas as gpd
import pandas as pd
from rasterio.features import rasterize

# definitions:

Data can be downloaded at: https://drive.google.com/drive/folders/1e3kjUWD-ikHo_TSmjCY86QyL-40R9xt3?usp=sharing. This data goes into ..\data\de_tol relative to the path-location of this notebook.

In [2]:
#input
data_path = Path(r"..\data\de_tol")
dem_tif = data_path.joinpath("dem_kockengen_20160905.tif")
wla_gpkg = data_path.joinpath("waterlevel_areas.gpkg")
gauges_gpkg = data_path.joinpath("gauges.gpkg")
rainfall_path = data_path.joinpath("rainfall")

# output
h2f_input_dir = data_path.joinpath("h2f_input")
h2f_input_dir.mkdir(parents=True, exist_ok=True)

# convert:
- DEM (GeoTIFF) to ascii: dem.asc
- Water_level_areas (GeoPackage) to acii: init_wl.asc
- Friction (constant_value) to ascii: friction.asc

### read the original DEM

In [3]:
with rasterio.open(dem_tif) as src:
    profile = src.profile
    data = src.read(1)

profile["driver"] = "AAIGrid"

### Original DEM to ESRI ASCII

In [4]:
with rasterio.open(h2f_input_dir.joinpath("dem.asc"), "w+", **profile) as dst:
    dst.write(data, 1)

### Water-level-areas to initial water-level ascii

We burn water-levels in the following order:
1. VASTPEIL (fixed level) as default
2. ZOMERPEIL (summer level) if 1 is nan
3. (ONDERPEIL + BOVENPEIL) / 2 if 2 is still nan

In [ ]:
gdf = gpd.read_file(wla_gpkg, bbox=src.bounds)
gdf.loc[:, "LEVEL"] = gdf["VASTPEIL"]
gdf.loc[gdf["LEVEL"].isna(), "LEVEL"] = gdf["ZOMERPEIL"]
gdf.loc[gdf["LEVEL"].isna(), "LEVEL"] = (gdf["BOVENPEIL"] + gdf["ONDERPEIL"]) / 2

data = data * 0 + profile["nodata"]
shapes = ((row["geometry"], row["LEVEL"]) for _, row in gdf.iterrows())
rasterize(shapes, out=data, out_shape=src.res, transform=profile["transform"])

with rasterio.open(h2f_input_dir.joinpath("init_wl.asc"), "w+", **profile) as dst:
    dst.write(data, 1)

C:\Anaconda3\envs\work\lib\site-packages\geopandas\geodataframe.py:577: RuntimeWarning: Sequential read of iterator was interrupted. Resetting iterator. This can negatively impact the performance.
  for feature in features_lst:


### Friction to ASCII

For now we apply a fixed manning roughness coefficient of 0.04 for the entire grid: https://www.engineeringtoolbox.com/mannings-roughness-d_799.html. Later we can vary with landuse/landcover.

In [ ]:
data = data * 0  + 0.04

with rasterio.open(h2f_input_dir.joinpath("friction.asc"), "w+", **profile) as dst:
    dst.write(data, 1)

### Rainfall ASCII to CSV

Sample rainfall ASCII to series to a pandas dataframe and save it as csv

In [ ]:
rainfall_ascs = rainfall_path.glob("*.ASC")
gdf = gpd.read_file(gauges_gpkg)
xy = [(geom.x, geom.y) for geom in gdf["geometry"]]
cols = ["datetime"] + [i for i in gdf["ID"]]
df = pd.DataFrame(columns=cols)

for idx, asc in enumerate(rainfall_ascs):
    datetime = pd.to_datetime(asc.stem, format="NSL_%Y%m%d_%H")
    with rasterio.open(asc) as src:
        values = [val[0] for val in src.sample(xy)]
        df.loc[idx] = [datetime] + values
        
df.set_index("datetime", inplace=True)
df.to_csv(h2f_input_dir.joinpath("rainfall.csv"),  float_format='%.2f')